In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForMaskedLM
import plotly.io as pio
from umap import UMAP
import hdbscan

c:\Users\Pessoal\Documents\Unifesp\Aries\aries_topic_selector\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from bertopic.representation import MaximalMarginalRelevance

Mostramos o diretório do arquivo CSV e definimos o dataframe utilizando a coluna de tweets limpos

In [17]:
nome_da_pasta = "resistencia_bacteria" # Sua variável para o nome da pasta
caminho_base = r'C:\Users\Pessoal\Documents\Unifesp\Aries\aries_topic_selector\dados\dadosTwitter\CSV'
nome_do_arquivo = 'tweets_processados.csv'


caminho_arquivo = fr'{caminho_base}\{nome_da_pasta}\{nome_do_arquivo}'
tweets_df = pd.read_csv(caminho_arquivo)
tweets_df = tweets_df[tweets_df["language"] == "pt"]


In [18]:
tweets_df.dropna(axis='columns', inplace=True)
tweets = tweets_df["tokens"].tolist()

### Usar dados do G1

In [12]:
caminho_arquivo = r'C:\Users\Pessoal\Documents\Unifesp\Aries\aries_topic_selector\dados\dadosG1\bacteria\news_processados.csv'
tweets_df = pd.read_csv(caminho_arquivo)
tweets_df.dropna(axis='columns', inplace=True)
tweets = tweets_df["tokens"].tolist()

# BERTimbau Base para Topic Modeling

Primeiro definimos o modelo de embedding, no caso, o BERTimbau

In [5]:
embedding_model = SentenceTransformer("neuralmind/bert-base-portuguese-cased")

No sentence-transformers model found with name neuralmind/bert-base-portuguese-cased. Creating a new one with mean pooling.


# Configurando o UMAP
técnica de redução de dimensionalidade. Ele pega seus embeddings de alta dimensão e os projeta em um espaço de menor dimensão enquanto tenta preservar a estrutura local e global dos dados

- n_neighbors: Controla o balanço entre a preservação da estrutura local versus global dos dados.
    Valores menores (por exemplo, 2 a 10) fazem com que o UMAP se concentre mais na estrutura local dos dados, resultando em clusters mais apertados e potencialmente mais tópicos, mas que podem ser muito específicos.
    Valores maiores (por exemplo, 15 a 50 ou mais) fazem com que o UMAP preserve mais a estrutura global, resultando em clusters mais dispersos e potencialmente menos tópicos, mas mais genéricos.
    Um bom ponto de partida é geralmente entre 10 e 15.
- n_components: A dimensão para a qual os embeddings serão reduzidos.
    Para visualização, geralmente usamos n_components=2 ou 3.
    Para o agrupamento, usar n_components entre 5 e 10 é comum. Um número maior de componentes pode preservar mais informações dos embeddings, mas pode tornar o agrupamento mais difícil para o HDBSCAN se houver muito "ruído".
- metric: A métrica de distância usada para calcular a proximidade entre os pontos nos embeddings.
    'cosine' é uma escolha comum e geralmente boa para embeddings de texto, pois mede a similaridade angular entre os vetores, o que é relevante para o significado semântico.



Treinamos os modelos nos Tweets já tokenizados

In [6]:
umap_model = UMAP(n_neighbors=5, #5 para mais clusters
                  n_components=5, 
                  metric='cosine',
                  random_state=42
                 )

# Configurando o HDBSCAN
É um algoritmo de agrupamento que agrupa pontos de dados com base na densidade. Ele é excelente para encontrar clusters de várias formas e tamanhos, e também consegue identificar "ruído" (documentos que não pertencem a nenhum tópico).

- min_cluster_size: O número mínimo de amostras em um cluster para ser considerado um cluster válido.
    Valores menores resultam em mais clusters e potencialmente mais ruído (tópicos com poucos documentos).
    Valores maiores resultam em menos clusters, mas mais densos e potencialmente mais robustos.

- cluster_selection_method: Define como os clusters são selecionados a partir da hierarquia.
    'eom' (default, "Excess of Mass") é geralmente a melhor opção, pois tende a produzir clusters mais equilibrados.
    'leaf' seleciona os clusters nas folhas da hierarquia, o que pode resultar em muitos clusters pequenos.

- metric='euclidean

In [7]:
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=10,
                                min_samples=5,
                                metric='euclidean',
                                cluster_selection_method='eom',
                                prediction_data=True 
                               )

# Inicializar o BERTopic com os modelos configurados e o parâmetro diversity

In [8]:
representation_model = MaximalMarginalRelevance(diversity=0.1, top_n_words=10)

topic_model = BERTopic(embedding_model=embedding_model,
                       language="portuguese",
                       umap_model=umap_model,
                       hdbscan_model=hdbscan_model,
                       representation_model=representation_model,
                       nr_topics="auto", # Ou um número fixo, ex: 50
                       min_topic_size=10 # Mantendo o padrão ou ajustando
                      )


##  Treinar o modelo

In [19]:
topics, probs = topic_model.fit_transform(tweets)

In [20]:
tweets_df["topic"] = topics
tweets_df["topic_prob"] = probs


In [21]:
topic_model.get_topic(1) # Exibir os tópicos encontrados
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,480,-1_tomar_infecção_tratamento_vírus,"[tomar, infecção, tratamento, vírus, medicamen...",[amiga gerar algum efeito colateral pessoa dep...
1,0,276,0_tomar_médico_infecção_remédio,"[tomar, médico, infecção, remédio, tratamento,...",[odeio ficar putario tomar médico receitar dps...
2,1,99,1_gonorreia_pneumoniae_antimicrobiano_brasil,"[gonorreia, pneumoniae, antimicrobiano, brasil...",[gene linhagem brasileiro aumentar infectar bo...
3,2,66,2_msm_nao_foder_amg,"[msm, nao, foder, amg, fuder, merda, caralho, ...",[mau fuder culpa dica thata tomir certinho est...
4,3,56,3_vírus_vacina_mutação_virus,"[vírus, vacina, mutação, virus, mutar, fungo, ...",[déiar achar entender analogia vírus criar vír...
5,4,37,4_mutação_enzima_célula_droga,"[mutação, enzima, célula, droga, antibiograma,...",[cientista identificar enzima usar combate car...
6,5,22,5_nao_direitinho_caralho_medo,"[nao, direitinho, caralho, medo, remédio, peri...",[andrea colocar noia cabeça tar perguntar ter ...
7,6,21,6_brasília_rio_multirresistente_brasil,"[brasília, rio, multirresistente, brasil, gast...",[direcinamento rdc anvisa equivocar foco probl...
8,7,13,7_acne_lactato_lachnospiraceae_diabete,"[acne, lactato, lachnospiraceae, diabete, isot...",[hereditariedade problema emocional produto ol...
9,8,13,8_neonatal_hospital_interditar_óbito,"[neonatal, hospital, interditar, óbito, matern...",[alto levar hospital universitário interditar ...


In [ ]:
tweets_df
tweets_df.to_excel(fr'{caminho_base}\{nome_da_pasta}\tweets_processados_vizualization.xlsx', index=False)
tweets_df.to_csv(caminho_arquivo, index=False)

In [14]:
topic_model.visualize_barchart(top_n_topics=14)

# Remover Categorias específicas do DataSet

In [ ]:
tweets_df = tweets_df[tweets_df["topic"] != 1]  # Remover tópicos de ruído
print(tweets_df)
tweets_df.to_csv(fr'{caminho_base}\{nome_da_pasta}\{nome_do_arquivo}', index=False)

In [45]:
topic_model.save("meu_modelo_bertopic")

2025-06-17 16:11:04,680 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


# Metricas de Avaliação

In [50]:
from bertopic import BERTopic
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import numpy as np
import gensim

In [ ]:


docs = tweets_df["tokens"].tolist()


topics = topic_model.get_topics()
top_n = 10  
topic_words = [[word for word, _ in topics[i][:top_n]] for i in topics if i != -1]

# Obter representações dos documentos tokenizados (pré-processados)
# Você pode usar diretamente os documentos processados se já tiver isso salvo
tokenized_docs = [doc.split() for doc in docs]

# Criar dicionário e corpus no formato Gensim
dictionary = Dictionary(tokenized_docs)
corpus = [dictionary.doc2bow(text) for text in tokenized_docs]

# Calcular Coerência
coherence_model = CoherenceModel(
    topics=topic_words,
    texts=tokenized_docs,
    dictionary=dictionary,
    coherence='c_v'
)
coherence_score = coherence_model.get_coherence()




In [116]:
# Calcular Diversidade
all_words = [word for topic in topic_words for word in topic]
unique_words = set(all_words)
diversity_score = len(unique_words) / len(all_words)



In [15]:
# Resultados
print(f"Coerência (C_V): {coherence_score:.4f}")
print(f"Diversidade: {diversity_score:.4f}")

NameError: name 'coherence_score' is not defined